<a href="https://colab.research.google.com/github/lusabo/fashionAI/blob/main/T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -r /content/results
!rm -r /content/t5-chatbot

In [ ]:
!pip install transformers[torch] datasets

In [ ]:
import os
import json
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from transformers.trainer_callback import TrainerCallback
import logging

In [ ]:
# Carregar o arquivo JSON
file_path = '/content/dataset.json'

with open(file_path, 'r') as file:
  data = json.load(file)

# Transformar os dados em um DataFrame
df = pd.DataFrame(data)

# Criar um Dataset do Hugging Face
dataset = Dataset.from_pandas(df)

In [ ]:
def preprocess_function(examples):
    inputs = [f"pergunta: {question}" for question in examples['pergunta']]
    targets = [f"resposta: {answer}" for answer in examples['resposta']]
    return {"input_text": inputs, "target_text": targets}

# Aplicar a função de preprocessamento
dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
# Verificar as primeiras linhas do Dataset para garantir que o preprocessamento está correto
print(f"Primeiros exemplos do Dataset após preprocessamento:\n{dataset[0]}")

In [ ]:
# Carregar o tokenizer e o modelo T5
model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    print(examples)
    model_inputs = tokenizer(examples["input_text"], max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(examples["target_text"], max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenizar o dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
# Verificar as primeiras linhas do Dataset tokenizado para garantir que a tokenização está correta
print(f"Primeiros exemplos do Dataset tokenizado:\n{tokenized_dataset[0]}")

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Definir os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Avaliar ao final de cada época
    learning_rate=3e-5,  # Taxa de aprendizado um pouco maior para garantir um aprendizado eficaz
    per_device_train_batch_size=8,  # Tamanho do lote moderado para equilibrar a memória e a eficiência
    per_device_eval_batch_size=8,  # Tamanho do lote de avaliação
    num_train_epochs=10,  # Mais épocas para garantir que o modelo tenha tempo suficiente para aprender
    weight_decay=0.01,  # Regularização para evitar overfitting
    save_strategy="epoch",  # Salvar o modelo ao final de cada época
    save_total_limit=2,  # Manter apenas os 2 últimos checkpoints
    fp16=True,  # Usar precisão mista para acelerar o treinamento e reduzir o uso de memória
)

# Data collator para garantir que as entradas estejam padronizadas durante o treinamento
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Dividir o dataset em treinamento e avaliação
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

In [ ]:
# Treinar o modelo
trainer.train()

In [ ]:
# Verificar e salvar o modelo e o tokenizer
output_dir = "./t5-chatbot"

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Carregar o modelo e o tokenizador treinados
model = T5ForConditionalGeneration.from_pretrained("./t5-chatbot")
tokenizer = T5Tokenizer.from_pretrained("./t5-chatbot")

def generate_answer(question, max_new_tokens=50, num_beams=3):
    input_text = f"pergunta: {question}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Gerar a resposta com os parâmetros ajustados
    outputs = model.generate(input_ids, max_new_tokens=512, num_beams=3, early_stopping=True)

    # Decodificar a resposta gerada
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [ ]:
# Testar com uma nova pergunta
print(generate_answer("Como usar calça jeans de forma estilosa?"))

In [ ]:
import pandas as pd
from transformers import T5Tokenizer
import matplotlib.pyplot as plt

# Carregar os dados
df = pd.read_json('/content/dataset.json')

# Carregar o tokenizer
model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Tokenizar os textos e obter o comprimento
input_lengths = [len(tokenizer.encode(text, truncation=False)) for text in df['pergunta']]
target_lengths = [len(tokenizer.encode(text, truncation=False)) for text in df['resposta']]

# Plotar a distribuição do comprimento dos tokens
plt.hist(input_lengths, bins=30, alpha=0.7, label='Perguntas')
plt.hist(target_lengths, bins=30, alpha=0.7, label='Respostas')
plt.xlabel('Número de Tokens')
plt.ylabel('Número de Exemplos')
plt.legend()
plt.title('Distribuição do Comprimento dos Tokens')
plt.show()

# Escolher um valor de max_length baseado na distribuição
max_length = 140  # Exemplo, ajuste conforme necessário

# Verificar a porcentagem de truncamento
truncated_inputs = sum([1 for length in input_lengths if length > 60]) / len(input_lengths) * 100
truncated_targets = sum([1 for length in target_lengths if length > max_length]) / len(target_lengths) * 100

print(f"Porcentagem de perguntas truncadas: {truncated_inputs:.2f}%")
print(f"Porcentagem de respostas truncadas: {truncated_targets:.2f}%")